In [2]:
import torch
import torch_geometric as pyg
import numpy as np
from torch_geometric.datasets import Planetoid
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data


In [52]:
{key: [1] for key in np.unique(r)}

{0: [1],
 1: [1],
 2: [1],
 3: [1],
 4: [1],
 5: [1],
 6: [1],
 7: [1],
 8: [1],
 9: [1],
 10: [1],
 11: [1]}

In [30]:
np.random.randn((11))

array([ 0.77079369, -0.00601667,  0.93852016,  1.26316455, -0.17326518,
        1.27018034, -0.07943724, -1.09252351, -1.76460828, -1.80166515,
        0.19705598])

In [64]:
l = np.array([[0, 1], [1, 4], [1, 5], [1, 6], [0, 2], [2, 7], [0, 3], [3, 8], [3, 9], [3, 10], [3, 11]]).transpose()
r = np.array([l[1, :], l[0, :]])
edge_index = torch.from_numpy(r)

# data = Planetoid("./dataset/", name='Cora')[0]
data = Data(edge_index=edge_index)

# train_loader = pyg.loader.NeighborSampler(edge_index, sizes=[1, 2])



# class NeighborSampler(Dataset):
#     def __init__(self, edges):
#         self.neighborhood_sampler = pyg.loader.NeighborSampler(edges, sizes=[1])

#     def __len__(self):
#         return len(self.neighborhood_sampler)

#     def __getitem__(self, idx):
#         return self.neighborhood_sampler.sample(idx)[1]


# data = NeighborSampler(edge_index)

loader = pyg.loader.NeighborLoader(
    data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[1] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=1,
    # input_nodes=data.train_mask,
)


# train_loader = pyg.loader.NeighborLoader(edge_index, num_neighbors=[1], batch_size=1, input_nodes=None)
                            
                               

In [65]:
sampled_data = next(iter(loader))
sampled_data

Data(edge_index=[2, 2], batch_size=1)

Data(edge_index=[2, 2], batch_size=1)

In [19]:
Data??

Init signature:
Data(
    x: Optional[torch.Tensor] = None,
    edge_index: Optional[torch.Tensor] = None,
    edge_attr: Optional[torch.Tensor] = None,
    y: Optional[torch.Tensor] = None,
    pos: Optional[torch.Tensor] = None,
    **kwargs,
)
Source:        
class Data(BaseData, FeatureStore, GraphStore):
    r"""A data object describing a homogeneous graph.
    The data object can hold node-level, link-level and graph-level attributes.
    In general, :class:`~torch_geometric.data.Data` tries to mimic the
    behaviour of a regular Python dictionary.
    In addition, it provides useful functionality for analyzing graph
    structures, and provides basic PyTorch tensor functionalities.
    See `here <https://pytorch-geometric.readthedocs.io/en/latest/notes/
    introduction.html#data-handling-of-graphs>`__ for the accompanying
    tutorial.

    .. code-block:: python

        from torch_geometric.data import Data

        data = Data(x=x, edge_index=edge_index, ...)

        # Add

In [24]:
def new_degree(edge_index, num_nodes, out=True):
    return pyg.utils.degree(edge_index[0 if out else 1], num_nodes)   


In [28]:
len(np.unique(r))

12

In [32]:
1 / new_degree(edge_index, len(np.unique(r)), False)

tensor([0.3333, 0.3333, 1.0000, 0.2500,    inf,    inf,    inf,    inf,    inf,
           inf,    inf,    inf])

In [ ]:
def proba_bfs(A, nodes, p, k_hop):

    nodes_to_be_explored = [**nodes]
    final_neighbors = []
    k = 0

    for k in range(k_hop):
        while nodes_to_be_explored:
            cur_node = nodes_to_be_explored.pop()
            neighbors = torch.nonzero(A[cur_node])

            indices = torch.randperm(len(neighbors))[:10]

            final_neighbors.extend(neighbors[indices])


In [3]:
from ogb.linkproppred import PygLinkPropPredDataset
from torch_geometric.data import DataLoader

dataset = PygLinkPropPredDataset(name = "ogbl-ppa") 

split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
graph = dataset[0] # pyg graph object containing only training edges


/home/jrm28/.conda/envs/gelato/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
data = graph

In [15]:
import torch
import numpy as np
import random
from torch_geometric.datasets import Planetoid, Amazon, PPI
from torch_geometric.utils import negative_sampling, add_self_loops, train_test_split_edges, k_hop_subgraph
from tqdm import tqdm

def set_random_seed(random_seed):
    """
    Set the random seed.
    :param random_seed: Seed to be set.
    """
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)


def split_dataset(data, valid_ratio=0.05, test_ratio=0.1, random_seed=0):
    """
    Split the edges/nonedges for biased training, full training, (full) validation and (full) testing.

    :param data: PyG dataset data.
    :param valid_ratio: ratio of validation edges.
    :param test_ratio: ratio of test edges.
    :param random_seed: random seed for the split.
    :return: edge splits
    """

    set_random_seed(random_seed)
    n = data.num_nodes

    print(data)
    split_data = train_test_split_edges(data, valid_ratio, test_ratio)
    split_edge = {'biased_train': {}, 'valid': {}, 'test': {}, 'train': {}}

    # Biased training with negative sampling.
    split_edge['biased_train']['edge'] = split_data.train_pos_edge_index.t()
    edge_index, _ = add_self_loops(split_data.train_pos_edge_index)  # To avoid negative sampling of self loops.
    split_data.train_neg_edge_index = negative_sampling(
        edge_index, num_nodes=split_data.num_nodes,
        num_neg_samples=split_data.train_pos_edge_index.size(1))
    split_edge['biased_train']['edge_neg'] = split_data.train_neg_edge_index.t()

    # Full training with all negative pairs in the training graph (including validation and testing positive edges).
    split_edge['train']['edge'] = split_data.train_pos_edge_index.t()
    train_edge_neg_mask = torch.ones((n, n), dtype=bool)
    train_edge_neg_mask[tuple(split_edge['train']['edge'].t().tolist())] = False
    train_edge_neg_mask = torch.triu(train_edge_neg_mask, 1)
    split_edge['train']['edge_neg'] = torch.nonzero(train_edge_neg_mask)

    # Full validation with all negative pairs in the training graph (including testing positive edges, excluding validation positive edges).
    split_edge['valid']['edge'] = split_data.val_pos_edge_index.t()
    valid_edge_neg_mask = train_edge_neg_mask.clone()
    valid_edge_neg_mask[tuple(split_edge['valid']['edge'].t().tolist())] = False
    split_edge['valid']['edge_neg'] = torch.nonzero(valid_edge_neg_mask)

    # Full testing with all negative pairs in the training graph (excluding validation and testing positive edges).
    split_edge['test']['edge'] = split_data.test_pos_edge_index.t()
    test_edge_neg_mask = valid_edge_neg_mask.clone()
    test_edge_neg_mask[tuple(split_edge['test']['edge'].t().tolist())] = False
    split_edge['test']['edge_neg'] = torch.nonzero(test_edge_neg_mask)

    return split_edge



def compute_edges(split_edge):
    """
    Compute the train, valid, and test edges based on edge split.

    :param split_edge: edge split.
    :return: train_edges_pos, train_edges_neg, valid_edges, valid_true, test_edges, test_true
    """

    # Train edges.
    train_edges_pos = split_edge['train']['edge']
    train_edges_pos = train_edges_pos[train_edges_pos[:, 0] < train_edges_pos[:, 1]]  # Only include upper triangle.
    train_edges_neg = split_edge['train']['edge_neg']

    # Valid edges.
    valid_edges = torch.vstack([split_edge['valid']['edge'], split_edge['valid']['edge_neg']])
    valid_true = torch.cat([torch.ones(split_edge['valid']['edge'].shape[0], dtype=int), torch.zeros(split_edge['valid']['edge_neg'].shape[0], dtype=int)])
    index = torch.randperm(valid_edges.shape[0])  # Shuffle edges for expected values of precision@k for ties.
    valid_edges = valid_edges[index]
    valid_true = valid_true[index]

    # Test edges.
    test_edges = torch.vstack([split_edge['test']['edge'], split_edge['test']['edge_neg']])
    test_true = torch.cat([torch.ones(split_edge['test']['edge'].shape[0], dtype=int), torch.zeros(split_edge['test']['edge_neg'].shape[0], dtype=int)])
    index = torch.randperm(test_edges.shape[0])
    test_edges = test_edges[index]
    test_true = test_true[index]

    return train_edges_pos, train_edges_neg, valid_edges, valid_true, test_edges, test_true


In [16]:

# data.edge_attr = torch.ones([data.edge_index.shape[1], 1], dtype=int)
split_edge = split_dataset(data)
data.edge_index = split_edge['train']['edge'].t()
data.edge_weight = data.train_pos_edge_attr
train_edges_pos, train_edges_neg, valid_edges, valid_true, test_edges, test_true = compute_edges(split_edge)

Data(num_nodes=576289, edge_index=[2, 42463862], x=[576289, 58])


/home/jrm28/.conda/envs/gelato/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


RuntimeError: [enforce fail at alloc_cpu.cpp:73] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 332109011521 bytes. Error code 12 (Cannot allocate memory)

In [21]:
split_edge['train']['edge'].shape

torch.Size([21231931, 2])